#### a notebook to save all the data from the efficiency cuts for future use
Applying the cuts every time is just stupid

In [2]:
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
                            # NOTE if you can't import IC stuff, its because of the
                            # above line
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import tables as tb
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit

from scipy.integrate import quad

from Tl_studies.functions import isaura_analyse as isa

In [13]:
def apply_cuts_raw(tracks, e_low_cut = 0.05, fid_lower_z = 20, fid_upper_z = 1195, fid_r_lim = 472, e_lower = 1.55, e_upper = 1.65):
    '''
    Apply all relevant cuts and spit out dataframe at the end
    The most barebones version, no efficiency calcs, it just spits out the dataframe
    '''


    # Low energy tracks, make this function work better. It seems busted currently
    low_e_cut_tracks = isa.remove_low_E_events(tracks, energy_limit = e_low_cut)
  
    # make fiducial cuts
    fiducial_rel = isa.fiducial_track_cut_2(low_e_cut_tracks, lower_z = fid_lower_z, upper_z=fid_upper_z, r_lim = fid_r_lim, verbose = False)

    # one track cuts 
    one_track_rel = isa.one_track_cuts(fiducial_rel, verbose = False)

    # overlapping blob cut
    ovlp_rel = isa.overlapping_cuts(one_track_rel)


    # energy cuts
    ecut_rel = isa.energy_cuts(ovlp_rel, lower_e = e_lower, upper_e = e_upper)

    # return it
    return ecut_rel


In [4]:
folder_path = '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1a/prod/isaura/'

# load in tracks
data = isa.load_tracks(folder_path)

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %


In [5]:
display(data)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.051154,121.454061,104,11063,2,368.675,160.675,334.439758,...,358.948175,381.376657,256.794897,408.658585,0.483891,0.015104,0.000000,9.857143,9.928571,9.619794
1,5080000,1,0.013352,9.928571,2,71,2,388.675,270.675,440.460083,...,442.236380,390.287922,283.647294,446.131930,0.013352,0.013352,0.013352,9.857143,9.928571,9.619794
2,5080002,0,1.768218,136.325163,128,11033,5,-98.325,361.675,587.473511,...,613.110772,-6.622819,382.940215,589.840154,0.162713,0.073897,0.000000,9.750000,9.571429,9.902647
3,5080002,1,0.309468,112.683992,31,1829,5,52.675,348.675,498.257599,...,502.310940,54.698632,349.802676,526.248103,0.162199,0.021946,0.000000,9.750000,9.571429,9.902647
4,5080002,2,0.019542,13.772242,2,4,5,18.675,351.675,555.981140,...,563.299622,27.644104,359.876147,555.981140,0.019542,0.019542,0.019542,9.750000,9.571429,9.902647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258315,4060708,1,0.121964,14.142136,3,1244,3,184.675,346.675,305.074768,...,309.084361,199.819036,352.583988,308.110573,0.121964,0.121964,0.121964,10.000000,10.000000,9.859631
258316,4060708,2,0.033485,10.000000,2,144,3,153.675,366.675,386.938629,...,389.045441,162.594082,366.980910,390.887885,0.033485,0.033472,0.033472,10.000000,10.000000,9.859631
258317,4060710,0,0.572125,44.565517,36,7243,1,315.675,294.675,522.397217,...,532.700813,331.038243,299.916212,527.865748,0.279909,0.256440,0.002211,9.833333,9.428571,8.912888
258318,4060712,0,0.888838,109.999957,76,6895,2,-6.325,415.675,335.498718,...,352.800524,25.058429,476.143700,418.546023,0.688630,0.026967,0.000000,9.500000,8.875000,9.673330


In [6]:
cut_data = apply_cuts_raw(data)

/home/e78368jw/Documents/NEXT_CODE/Tl_studies/quick_notebooks/../../Tl_studies/functions/isaura_analyse.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_low_E['numb_of_tracks'] = remove_low_E['event'].map(event_counts)


In [7]:
display(cut_data)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
184,5080188,0,1.596243,137.456298,113,11993,1,156.675,107.675,734.490601,...,789.447427,249.494267,184.766019,738.319437,0.511776,0.245868,0.0,10.000000,9.333333,9.592146
518,5080496,0,1.594183,154.743531,127,12870,1,147.675,-166.325,322.920868,...,348.181738,150.682122,-134.107884,329.234220,0.444847,0.443488,0.0,9.533333,9.200000,8.737142
606,5080582,0,1.595847,134.427703,165,12424,1,293.675,-43.325,683.036133,...,726.958593,344.200068,58.357780,714.376938,0.327431,0.169420,0.0,9.272727,9.615385,8.566711
674,5080676,0,1.594673,123.527479,117,12643,1,185.675,150.675,453.089264,...,466.743201,273.441110,157.500124,496.954274,0.612307,0.345949,0.0,9.416667,10.000000,9.292014
741,1740022,0,1.595029,246.160982,147,12240,1,186.675,-40.325,405.846893,...,498.364705,201.867704,58.954131,414.055713,0.492123,0.414989,0.0,10.000000,9.333333,9.967749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187495,4940584,0,1.598050,118.225238,144,12842,1,299.675,21.675,843.681885,...,909.920034,365.272391,95.558628,847.721470,0.475479,0.155838,0.0,9.100000,9.636364,8.917419
187630,4060092,0,1.611933,139.488392,138,11245,1,147.675,351.675,1015.907898,...,1023.885905,199.399870,392.845961,1111.865537,0.452174,0.263725,0.0,9.454545,9.100000,9.443298
187873,4060350,0,1.594714,193.786162,142,12044,1,43.675,21.675,412.145233,...,443.268241,95.273416,71.054266,541.965957,0.498915,0.433457,0.0,9.769231,9.000000,9.744980
187893,4060374,0,1.604224,110.101848,113,13741,1,231.675,227.675,516.088623,...,540.399202,301.620382,314.980297,538.400907,0.565798,0.366740,0.0,9.625000,9.300000,9.434460


In [8]:
# save cut data
cut_data.to_hdf('P_1A_10x10x10.h5', key = 'cut', mode = 'w')

In [9]:
test = pd.read_hdf('P_1A_10x10x10.h5')
display(test)
del test

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
184,5080188,0,1.596243,137.456298,113,11993,1,156.675,107.675,734.490601,...,789.447427,249.494267,184.766019,738.319437,0.511776,0.245868,0.0,10.000000,9.333333,9.592146
518,5080496,0,1.594183,154.743531,127,12870,1,147.675,-166.325,322.920868,...,348.181738,150.682122,-134.107884,329.234220,0.444847,0.443488,0.0,9.533333,9.200000,8.737142
606,5080582,0,1.595847,134.427703,165,12424,1,293.675,-43.325,683.036133,...,726.958593,344.200068,58.357780,714.376938,0.327431,0.169420,0.0,9.272727,9.615385,8.566711
674,5080676,0,1.594673,123.527479,117,12643,1,185.675,150.675,453.089264,...,466.743201,273.441110,157.500124,496.954274,0.612307,0.345949,0.0,9.416667,10.000000,9.292014
741,1740022,0,1.595029,246.160982,147,12240,1,186.675,-40.325,405.846893,...,498.364705,201.867704,58.954131,414.055713,0.492123,0.414989,0.0,10.000000,9.333333,9.967749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187495,4940584,0,1.598050,118.225238,144,12842,1,299.675,21.675,843.681885,...,909.920034,365.272391,95.558628,847.721470,0.475479,0.155838,0.0,9.100000,9.636364,8.917419
187630,4060092,0,1.611933,139.488392,138,11245,1,147.675,351.675,1015.907898,...,1023.885905,199.399870,392.845961,1111.865537,0.452174,0.263725,0.0,9.454545,9.100000,9.443298
187873,4060350,0,1.594714,193.786162,142,12044,1,43.675,21.675,412.145233,...,443.268241,95.273416,71.054266,541.965957,0.498915,0.433457,0.0,9.769231,9.000000,9.744980
187893,4060374,0,1.604224,110.101848,113,13741,1,231.675,227.675,516.088623,...,540.399202,301.620382,314.980297,538.400907,0.565798,0.366740,0.0,9.625000,9.300000,9.434460


In [10]:
folder_paths =  ['../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1a/prod/isaura/',
                '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_1b/prod/isaura/',
                '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_2a/prod/isaura/',
                '../LPR_rerun_voxel/isaura_10x10x10mm/PORT_2b/prod/isaura/']
h5_file_names = ['P1A_10x10x10.h5',
                 'P1B_10x10x10.h5',
                 'P2A_10x10x10.h5',
                 'P2B_10x10x10.h5']


for i in range(len(folder_paths)):
    print(h5_file_names[i])
    print("")
    # load data
    data = isa.load_tracks(folder_paths[i])

    # cut data
    cut_data = apply_cuts_raw(data)

    # save data
    cut_data.to_hdf(h5_file_names[i], key = 'cut', mode = 'w')

    # cleanup
    del data
    del cut_data

P1A_10x10x10.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %


/home/e78368jw/Documents/NEXT_CODE/Tl_studies/quick_notebooks/../../Tl_studies/functions/isaura_analyse.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_low_E['numb_of_tracks'] = remove_low_E['event'].map(event_counts)


P1B_10x10x10.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %
P2A_10x10x10.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %
P2B_10x10x10.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %


In [14]:
# proof of concept
proof = pd.read_hdf('P1A_10x10x10.h5')
display(proof.head(20))

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
184,5080188,0,1.596243,137.456298,113,11993,1,156.675,107.675,734.490601,...,789.447427,249.494267,184.766019,738.319437,0.511776,0.245868,0.0,10.000000,9.333333,9.592146
518,5080496,0,1.594183,154.743531,127,12870,1,147.675,-166.325,322.920868,...,348.181738,150.682122,-134.107884,329.234220,0.444847,0.443488,0.0,9.533333,9.200000,8.737142
606,5080582,0,1.595847,134.427703,165,12424,1,293.675,-43.325,683.036133,...,726.958593,344.200068,58.357780,714.376938,0.327431,0.169420,0.0,9.272727,9.615385,8.566711
674,5080676,0,1.594673,123.527479,117,12643,1,185.675,150.675,453.089264,...,466.743201,273.441110,157.500124,496.954274,0.612307,0.345949,0.0,9.416667,10.000000,9.292014
741,1740022,0,1.595029,246.160982,147,12240,1,186.675,-40.325,405.846893,...,498.364705,201.867704,58.954131,414.055713,0.492123,0.414989,0.0,10.000000,9.333333,9.967749
803,1740082,0,1.621295,232.570108,143,11521,1,-9.325,-335.325,191.680923,...,255.239925,67.675000,-163.325000,249.403687,0.457842,0.014532,0.0,9.700000,9.761905,9.968830
1269,1740572,0,1.610053,145.416054,150,12892,1,-101.325,-456.325,436.304749,...,522.938656,-90.355666,-372.274817,527.177016,0.448995,0.172687,0.0,9.000000,9.384615,9.442682
1512,3580184,0,1.596952,210.176919,136,12720,1,87.675,286.675,153.902756,...,214.041428,218.983909,291.827323,188.099396,0.496821,0.448385,0.0,9.857143,9.714286,9.726704
1673,3580384,0,1.608184,156.671782,138,12844,1,-132.325,83.675,967.594543,...,980.742616,-38.293017,199.577009,1025.428229,0.343598,0.258966,0.0,9.900000,9.437500,9.095669
1836,3580568,0,1.587940,72.108443,78,11981,1,-284.325,-63.325,227.352615,...,255.964403,-279.837385,-2.068621,245.422964,0.588401,0.306167,0.0,9.100000,9.857143,8.167691
